# Aeroelastic Optimization

## Introduction

The main goal of this project is to develop an open source software for perform an aeroelastic optimization on a established wing model. Starting from a FEM model and a CFD model of the wing the code should find the best geometry and properties for this wing minimizing or maximizing a set variable, like the global mass or the induced drag coefficient, but respecting some constraints setted in the start, like the stress criteria or the lift criteria.

## Softwares

As we said the goal is to develop an open source software, so for this purpose the choise for the main script code's language is python. Using the potential of python we can launch an entire optimization, and all the program needed for perform it ( FEM analysis, aerodynamic analysis, ...), just from one script.

<img src="img/softw.png" alt="Drawing" style="width: 400px;"/>

The script have written in python languange with the support of the openMDAO library for the optimization part. For the aerodinamic part, to get the aerodynamic load and properies we used Panair, with the support of the preprocessing tool Panin to write the input file for Panair starting with the value of variables defined in the script. To perform the structural analisys we used Nastran95.

To run the script is necessary to install correctly these software, see the installation guide.

## Optimization Problem

As we said in the introduction the goal is to develop a code unable to permorf a generic optimization starting from any given geometry. So the user can choose the design variables, the objective function and the constraints. To do that function have been written to communicate wich each other using the common variables; so during the process when a variables changes, performing the aerodynamic and structural analisys, we can see the effect of this change on the other variables.

The major function (in green in the figure) are:
* an [aerodymic function](http) that starting from the aerodynamyc parameters as wing properties (c,b,S,...), angle of attack $\alpha$ and flight properties ( M,$\rho$,V,...), can launch Panair to perform the analisys, and can read, from the output file the aerodynamic properties ( $C_D$,$C_L$,aerodynamic loads, ...);
* an [interpolation function](link) that starting from the aerodynamic mesh can transfer everything on the structural mesh, and vice versa;
* an [structural function](link) that starting from the structural parameters as geometry properties, material properties and loads, can launch Nastran95 to perform the analisys, and can read, from the output file the displacement of the nodes, the weight of the wing.

Let's see how the optimitazion loop is built:

<img src="img/xdc.png" alt="Drawing" style="width: 800px;"/>

As u can see the main loop is the optimization loop, that will evaluate the objective function( in this case mass and/or $C_{D_i}$), the constraints ( in this case $C_L$ and $\sigma$) for each change of the design variables ( in this case $\alpha$ and  the thickness of the QUAD elements ). There is also an internal loop, the MDA loop, that will evaluate the dispacement and the loads until convergence, in fact the aerodynamic forces depend on the the geometry of the wing, and they cause a deformation of the wing, that will change the aerodynamic forces, so in the MDA loop we are performing a loop of Panair-Nastran, that update itaration for iteration the displacement of the nodes in the Panair geometry file and the nodal force in the Nastran bdf file, until the convergerce it's guaranteed.

## Optimizer

We used for the optimizer the openMDAO library. For our optimization we can choose two type of optimizer:
* **SLSQP**, it's an optimizer gradient based, but for our problem we didn't calculate the gradient, so to use it we can use the finite differnce method; that's mean that in the start the optimizer will change the design variable to a set step and evaluate the gradient to launch the optimization, until the process it can relaunch the finite difference step to re-evaluate that gradient. In the next figure there is an example of SLSQP optimization where u can see the step on the thickness vector where he evaluate the gradient.
<img src="img/sls.png" alt="Drawing" style="width: 400px;"/>
NOTE: Be carefull on the lenght of the step, because if the step is too little it cannot recognize change, that's why nastran98 use a 8float architecture, so when when the program convert the varibles in 8float number for nastran u can lose the information, and the result of the static analisys will be the same!

* **COBYLA**, it's an optimizer gradient free, and for our case where we didn't calculted the gradients it's the best soluction. The best results we obained is with this optimizer, but as u can see in the next figure the optimization process can be longer and full of oscillation. Using COBYLA there is another problem, how it's develop it doesn't respect the limits of the design variables, so the final result can be physically impossible; we solved that problem, as we explain in the dedicate chapter, using the design variables limit as constraint.
<img src="img/cobyla.png" alt="Drawing" style="width: 400px;"/>

## Input Files

To run our script it's necessary to upload some sample file:
* a .wgs file, containing the information about the aerodynamic mesh, all the coordinates of the nodes of the mesh;
<img src="img/aero.png" alt="Drawing" style="width: 400px;"/>
* a .inp file, input file for Nastran95, in the folder there is the sample file to start the process, for the CRM wing it presents 12 different thickness section of the QUAD element, instead for the goland wing it present 17 different thickness section;
<img src="img/fem.png" alt="Drawing" style="width: 400px;"/>
* a .aux file, input file for panin that generate the input file for panair.

## Design Variables, constraint and objective

In our script we set as design varibles the angle of attack $\alpha$ and the thicknesses of the QUAD elements. 

Instead we set a lift constraint $con_L = C_L - \frac{W}{(\frac{1}{2}*\rho_a*V^2*S_w)}>0$ and a stress constraint $ con_{\sigma}= max\{\sigma_{VonMises}\}-\sigma_{yield}<0$. Using the COBYLA optimiter to be sure that the limits on the design variables will be respected we put also that limits as constraints $con_{t_{max}}(i)= t(i)-t_{max}<0$ and $con_{t_{min}}(i)= t(i)-t_{min}>0$ for each different thickness section.

As objective we set in our test cases the mass of the wing and the induced drag coefficient.